## Assignment 3: Image Classification with Convolutional Neural Networks (CNNs)

Group: Group 1<br>
Name: Shaun Bisset, Qichun Yu

### Part 1: Building a Multiclass Classifier Based on the Fashion MNIST Dataset 
Main author: Shaun Bisset

1. Create a Jupyter notebook and import the necessary libraries. Import the Fashion MNIST dataset from TensorFlow using:


In [ ]:
#Q1 import libraries
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#set base image parameters
batch_size = 16
img_height = 28
img_width = 28

np.random.seed(8)
tf.random.set_seed(8)

2. Describe the dataset. How many images are there and in the training and testing sets? What size are they? Are the images colour, grayscale or black and white? List the values that the label can take. You can read more about this dataset on TensorFlow's website here: https://www.tensorflow.org/datasets/catalog/mnist

Q2 Describe data <br/>
Train data consists of 60000 greyscale images 28px X 28px showing 10 different types of clothing including T-Shirt, Trousers, Pullover hoodie, Dress, Coat,Sandal, Shirt, Sneaker, Bag, Ankle boot. The test data set is the same types of images, color, size etc. though there are only 10000 of them. The label is a single integer from 0-9 each representing one of the clothing items listed above. 

3. Load the Fashion MNIST dataset using fashion_mnist.load_data() and save the results to (features_train, label_train), (features_test, label_test).

In [ ]:
#Q3 train test split
(features_train, label_train), (features_test, label_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


4. Print out the shape of features_train and features_test. Reshape these to add a new dimension corresponding to the number of colours, so that they are compatible with the model.

In [ ]:
#Q4 reshape to add 3th demention
print('Train Feature list\n',features_train.shape)
print('Test Feature list\n',features_test.shape)

features_train = features_train.reshape(60000, 28, 28, 1)
features_test = features_test.reshape(10000, 28, 28, 1)

Train Feature list
 (60000, 28, 28)
Test Feature list
 (10000, 28, 28)


5. Create a data generator with the following data augmentation for training:

In [ ]:
#Q5 creating training data generator
dataGeneratorTrain = ImageDataGenerator(
                                        rescale=1./255, 
                                        rotation_range=40, 
                                        width_shift_range=0.1, 
                                        height_shift_range=0.1, 
                                        shear_range=0.2, 
                                        zoom_range=0.2, 
                                        horizontal_flip=True, 
                                        fill_mode='nearest')

dataGeneratorTrain = dataGeneratorTrain.flow(features_train, label_train, batch_size=batch_size)

6. Create another data generator for testing that only has rescaling.

In [ ]:
#Q6 creating testing data generator
dataGeneratorTest = ImageDataGenerator(rescale=1./255)

dataGeneratorTest = dataGeneratorTest.flow(features_train, label_train, batch_size=batch_size)

7. Create a neural network architecture with the following layers: a convolutional layer with Conv2D(64, (3,3), activation='relu') followed by MaxPooling2D(2,2); a convolutional layer with Conv2D(64, (3,3), activation='relu') followed by MaxPooling2D(2,2); a flatten layer; a fully connected layer with Dense(128, activation='relu'); a fully connected layer with Dense(10, activation='softmax').

In [ ]:
#Q7 network architechture
model = tf.keras.Sequential([
                              layers.Conv2D(64, (3,3), activation='relu'),
                              layers.MaxPooling2D(2,2),
                              layers.Conv2D(64, (3,3), activation='relu'),
                              layers.MaxPooling2D(2,2),
                              layers.Flatten(),
                              layers.Dense(128, activation='relu'),
                              layers.Dense(10, activation='softmax')])

8. Compile the model using an Adam optimizer with a learning rate of 0.001.

In [ ]:
#Q8 adamizer
optimizer = tf.keras.optimizers.Adam(0.001)

9. Train the model and evaluate it on the testing set. Comment on the results.

In [ ]:
#Q9.1
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
#9.2
model.fit_generator(dataGeneratorTrain, steps_per_epoch=len(features_train) // batch_size, epochs=5, validation_data=dataGeneratorTest, validation_steps=len(features_test) // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
3750/3750 [==============================] - 138s 36ms/step - loss: 0.8150 - accuracy: 0.6960 - val_loss: 0.5832 - val_accuracy: 0.7729
Epoch 2/5
3750/3750 [==============================] - 135s 36ms/step - loss: 0.5986 - accuracy: 0.7737 - val_loss: 0.5034 - val_accuracy: 0.8099
Epoch 3/5
3750/3750 [==============================] - 134s 36ms/step - loss: 0.5377 - accuracy: 0.7974 - val_loss: 0.5521 - val_accuracy: 0.7984
Epoch 4/5
3750/3750 [==============================] - 132s 35ms/step - loss: 0.5026 - accuracy: 0.8103 - val_loss: 0.4139 - val_accuracy: 0.8400
Epoch 5/5
3750/3750 [==============================] - 133s 36ms/step - loss: 0.4753 - accuracy: 0.8218 - val_loss: 0.4010 - val_accuracy: 0.8447


An accuracy of 82% was acheived as per expected in the assignment instructions.

### Part 2: Fruit Classification with Transfer Learning
Main author: Qichun Yu

1. Create a new notebook and import the necessary libraries, including pathlib.

In [ ]:
import pathlib

2. Import the dataset and unzip the file using TensorFlow. Use the following code to download and unzip the Fruits360 dataset, and setup the training and validation paths:

In [ ]:
# download from github, locate the fruits360_filtered and store in the path variable
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Activity3.02/fruits360.zip'
zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=file_url, extract=True)
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'
train_dir = path / 'Training'
validation_dir = path / 'Test'

82231296/82220233 [==============================] - 0s 0us/step


3. Describe the dataset. How many images are there and in the training and testing sets? What size are they? Are the images colour, grayscale or black and white?

In [ ]:
# use read function to read readme file
readme = path / 'fruits-360-readme.md'
with open(readme) as readme:
    print(readme.read())

# Fruits-360: A dataset of images containing fruits and vegetables

## Version: 2019.09.21.0 ##

A high-quality, dataset of images containing fruits and vegetables. The following fruits and vegetables are included: 
Apples (different varieties: Crimson Snow, Golden, Golden-Red, Granny Smith, Pink Lady, Red, Red Delicious), Apricot, Avocado, Avocado ripe, Banana (Yellow, Red, Lady Finger), Beetroot Red, Blueberry, Cactus fruit, Cantaloupe (2 varieties), Carambula, Cauliflower, Cherry (different varieties, Rainier), Cherry Wax (Yellow, Red, Black), Chestnut, Clementine, Cocos, Dates, Eggplant, Ginger Root, Granadilla, Grape (Blue, Pink, White (different varieties)), Grapefruit (Pink, White), Guava, Hazelnut, Huckleberry, Kiwi, Kaki, Kohlrabi, Kumsquats, Lemon (normal, Meyer), Lime, Lychee, Mandarine, Mango (Green, Red), Mangostan, Maracuja, Melon Piel de Sapo, Mulberry, Nectarine (Regular, Flat), Nut (Forest, Pecan), Onion (Red, White), Orange, Papaya, Passion fruit, Peach (different var

The Fruits-360 is an image dataset that contains fruits and vegetables (such as avocados, bananas, blueberry, cantaloupe, cherry and so on). The dataset has 120 classes of fruits and vegetables. From the fruits-360-readme.md file, the total number of images are 82,213. There are 61,488 images in the training sets and 20,622 images in the testing sets. The image size is 100 * 100 pixels. Images are colour with a white sheet of paper as a background. From the fruit-360 file that we downloaded, there are 11,398 images in the training folder and 4,752 images in the testing folder. 

4. Create a data generator for 100 by 100 pixel images, with the following data augmentation for training:

In [ ]:
# Create a data generator
train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           rotation_range=40, 
                                           width_shift_range=0.1, 
                                           height_shift_range=0.1, 
                                           shear_range=0.2, 
                                           zoom_range=0.2, 
                                           horizontal_flip=True, 
                                           fill_mode='nearest')

5. Create another data generator for testing 100 by 100 pixel images that only has rescaling.

In [ ]:
# Create data generator for testing
validation_image_generator = ImageDataGenerator(rescale=1./255)

6. Load a pretrained VGG16 model from TensorFlow to be your base model. Specify weights='imagenet'and include_top=False. You will also need to know the image dimensions and number of channels (colours). Set this base model to be non-trainable using the .trainable attribute. Print a summary of the base model. How many parameters are there, and how many are trainable?

In [ ]:
from tensorflow.keras.applications import VGG16

# create vaiables to store values for batch_size, image height and width, and channel
batch_size = 16
img_height = 100
img_width = 100
channel = 3

# Instantiate a VGG16 model
base_model = VGG16(input_shape=(img_height, img_width, channel), weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# create a data generator called train_data_gen
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, directory=train_dir, target_size=(img_height, img_width))
# create a data generator called val_data_gen
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size, directory=validation_dir, target_size=(img_height, img_width))



Found 11398 images belonging to 120 classes.
Found 4752 images belonging to 120 classes.


In [ ]:
# create two variables that will get the number of images for the training and validation sets
total_train = 11398
total_val = 4752

In [ ]:
# Set this model to non-trainable
base_model.trainable =  False

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

There are 14,714,688 parameters are there, but zero are trainable as we have frozen all the layers of this model. 

7. Create a new model using tf.keras.Sequential() by adding the base model to the following layers: a flattening layer Flatten() , a fully connected layer Dense(1000, activation='relu') and a fully connected layer Dense(120, activation='softmax').

In [ ]:
# create new model using Sequential
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(120, activation='softmax')
])

8. Compile the model using an Adam optimizer with a learning rate of 0.001, categorical cross-entropy for the loss, and use accuracy as a metric.

In [ ]:
# instantiate using adam optimizer with a learning rate 0.001
optimizer = tf.keras.optimizers.Adam(0.001)

9. Train the model. Evaluate the model on the testing set. Comment on the results.

In [ ]:
# compile the neural network model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 1000)              4609000   
                                                                 
 dense_3 (Dense)             (None, 120)               120120    
                                                                 
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# using fit_generator() to fit the neural network model with a epochs=5
model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


712/712 [==============================] - 1670s 2s/step - loss: 1.9965 - accuracy: 0.5166 - val_loss: 0.9953 - val_accuracy: 0.7376
Epoch 2/5
712/712 [==============================] - 1692s 2s/step - loss: 0.6607 - accuracy: 0.8074 - val_loss: 0.5755 - val_accuracy: 0.8260
Epoch 3/5
712/712 [==============================] - 1667s 2s/step - loss: 0.4201 - accuracy: 0.8715 - val_loss: 0.5731 - val_accuracy: 0.8359
Epoch 4/5
712/712 [==============================] - 1663s 2s/step - loss: 0.3341 - accuracy: 0.8935 - val_loss: 0.5427 - val_accuracy: 0.8590
Epoch 5/5
712/712 [==============================] - 1664s 2s/step - loss: 0.2825 - accuracy: 0.9104 - val_loss: 0.4005 - val_accuracy: 0.8761


After training with five epochs, we achieved an accuracy score of 0.9104 for the training set and 0.8761 for the testing set. The score indicates the result of the training is remarkable. 